In [1]:
import shapely 
import phidl.geometry as pg
from phidl.path import Path,CrossSection
import numpy as np
import matplotlib.pyplot as plt
from phidl.quickplotter import quickplot2 as qp
import utilities as util
%matplotlib qt

In [2]:
elec_diam=25
width_ring = 20
n_arms = 8
n_rings = 5
width_lastring = 50
l_arm = 1000
width_arm_end = 125
n_electrodes_per_ring = [1,8,12,16,24]


In [3]:
def arm(l_arm,elec_diam,width2):
    return pg.taper(length=l_arm,width1=0.7*elec_diam,width2=width2)

device=pg.Device()
elec_pos_xy=[]

for i in range(n_arms):
    device<< arm(l_arm,elec_diam,125).rotate(360//8*i)
    if i<n_rings:
        if i==n_rings-1:
            width=width_lastring
        else:
            width=width_ring
        device<< pg.ring(radius=(i+1)*l_arm//n_rings, width=width)


device<<pg.circle(radius=elec_diam/2)
for ring_idx,n_elecs in enumerate(n_electrodes_per_ring):
    angle = np.linspace(0,2*np.pi,n_elecs+1)[:-1] + np.pi/n_elecs
    r=l_arm//n_rings*(ring_idx)
    x_l,y_l = np.cos(angle)*r,np.sin(angle)*r
    for x_,y_ in zip(x_l,y_l):
        device<<pg.circle(radius=elec_diam/2+10).movex(x_).movey(y_)
        elec_pos_xy.append([x_,y_])

In [4]:
qp(device)

In [5]:
sh_dev = util.phidl_to_shapely(device).buffer(5).buffer(-5)

In [6]:
router = util.RoutingLayout(sh_dev,border_tolerance=5)

In [7]:
router.lay_down_lines(n_lines=7,gap_um=5)

In [8]:
elec_boundaries=[]
elec_additional_pts=[]

for x,y in elec_pos_xy:
    elec_boundary=util.phidl_to_shapely(pg.circle(radius=elec_diam/4).movex(x).movey(y)).boundary
    t=np.arctan2(y,x)
    r=np.sqrt(x**2+y**2)
    p1 = shapely.Point( (r+25)*np.cos(t),(r+25)*np.sin(t))
    p2 = shapely.Point( (r-25)*np.cos(t), (r-25)*np.sin(t))
    extra_points = shapely.MultiPoint( [p1,p2] )
    elec_boundaries.append(elec_boundary)
    elec_additional_pts.append(extra_points)

In [9]:
router.add_electrodes(elec_boundaries,elec_additional_pts,type='electrode')

In [10]:
n_total_elecs = len(elec_pos_xy)
pad_boundaries=[]
pad_extra_points=[]
for i in range(n_total_elecs):
    angle = 2*np.pi/(n_total_elecs+1)*i
    r = l_arm + width_lastring/2
    x,y=np.cos(angle)*r,np.sin(angle)*r
    boundary = shapely.Point(x,y).buffer(1).boundary
    extra_points = shapely.MultiPoint( [shapely.Point( (r-width_lastring)*np.cos(angle),(r-width_lastring)*np.sin(angle))])
    pad_boundaries.append(boundary)
    pad_extra_points.append(extra_points)
router.add_electrodes(pad_boundaries,pad_extra_points,type='pad')

In [11]:
router.route()

Routing successful


In [12]:
lines=util.multilinestrings_to_phidl(shapely.unary_union(router.all_elec_pad_routes_smooth),width=2,layer=1)
final_device=pg.Device()
final_device<<device
electrodes=pg.Device()
for x,y in elec_pos_xy:
    electrodes<<pg.circle(radius=10,layer=1).movex(x).movey(y)
final_device<<electrodes
final_device<<lines
final_device<<pg.boolean(device,electrodes,operation='A-B',layer=2)

DeviceReference (parent Device "boolean", ports [], origin (0, 0), rotation 0, x_reflection False)

In [13]:
final_device.write_gds('Example_McDonald.gds')

'Example_McDonald.gds'

In [14]:
qp(final_device)